In [1]:
import itertools
from tqdm import tqdm_notebook

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

from IPython.display import Image, SVG

%matplotlib inline

In [2]:
import json
with open('C:\\Users\\Xiaomi\\JN\\mail.ru\\homework4\\cosmo_content_storage_final_cut.jsonl', 'rb') as json_file:
    json_list_data = [eval(line.decode("utf-8"))  for line in json_file.readlines()]
with open('C:\\Users\\Xiaomi\\JN\\mail.ru\\homework4\\cluster_final_cut_train.json', 'rb') as json_file:
    json_list_target = [eval(line.decode("utf-8"))  for line in json_file.readlines()]
    
    

In [3]:
fullData = pd.DataFrame(json_list_data)
fullTargetData = pd.DataFrame(json_list_target)
fullTextData = fullData.drop(columns=['ts', 'url'])
fullTextData['text'] = fullTextData.description + " " + fullTextData.title
del fullTextData['description']
del fullTextData['title']
# textData = fullTextData.drop(columns=['description'])
# descData = fullTextData.drop(columns=['title'])
fullData.head()

,description,doc_id,title,ts,url
0,У России есть необходимые конкурентные преимущ...,1000029981939875422,Новак заявил о способности России стать лидеро...,1575625264,https://vz.ru/news/2019/12/6/1012187.html
1,По делу об убийстве главы ингушского центра «Э...,1000103932845397396,"СК отчитался о поимке банды, причастной к убий...",1575634419,https://meduza.io/news/2019/12/06/sk-otchitals...
2,Президент Азербайджана Ильхам Алиев и первый в...,1000115462666782749,Ильхам Алиев и Мехрибан Алиева посетили выстав...,1575447101,http://www.vestikavkaza.ru/material/283355
3,Проблемы и вызовы языковой политики анализируе...,1000129039443894284,Язык и идеология,1575767444,http://krizis-kopilka.ru/archives/70325
4,Отношения России и Турции не являются альтерна...,1000131983536261699,Эрдоган: отношения Турции с РФ не альтернативн...,1575359865,https://radiosputnik.ria.ru/20191203/156189583...


# Формирование marked и unmarked датасетов 

In [4]:
markDataFull = pd.DataFrame(columns=list(fullTextData.columns) + ['mark'])
for id in fullTargetData.columns:
    t = fullTextData.loc[fullTextData['doc_id'] == int(id)].iloc[0]
    t1 = t.append(pd.Series([fullTargetData[id].iloc[0]], index=["mark"]))
    markDataFull.loc[t.name] = t1

In [5]:
indexMarkedData = markDataFull.index.values.tolist()
indexAllData = fullTextData.index.values.tolist()
indexUnmarkedData = [item for item in indexAllData if item not in indexMarkedData]


In [6]:
unmarkedData = fullTextData.iloc[indexUnmarkedData]

In [7]:
nans = fullTextData.text[fullTextData.text.isna()]
for index in nans.index:
    fullTextData.text[index] = ""

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Обработка

In [8]:
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import re
import string
stemmerRus = SnowballStemmer("russian") 
stemmerEng = SnowballStemmer("english") 
stop_words = set(stopwords.words('russian'))
stop_words = stop_words.union(set(stopwords.words('english')))
wl = WordNetLemmatizer()

def preprocessing(text):
    temp = text.lower()
    temp = re.sub(r'\d+', '', temp)
    temp = temp.translate(str.maketrans('','', string.punctuation)).strip()
    tokens = word_tokenize(temp)
    temp = [wl.lemmatize(i) if not bool(re.search('[а-яА-Я]', i)) else stemmerRus.stem(i) for i in tokens if not i in stop_words]
    temp = [word if len(word)>2 else '' for word in temp]
    return ' '.join(temp)

In [9]:
preprocessing("жили у бабуси hello books a ? - [ ] . 123 два веселых гуся \"  один белый другой серый")

'жил бабус hello book весел  бел сер'

# Токенизация

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(preprocessor=preprocessing)
vectorizer.fit(fullTextData.text)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessing at 0x000001B924D8CD08>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)

In [11]:
fullTextData.text = [preprocessing(text) for text in fullTextData.text]
matrixFullText = vectorizer.transform(fullTextData.text)
matrixFullText.shape

(88368, 75869)

In [12]:
nans = unmarkedData.text[unmarkedData.text.isna()]
for index in nans.index:
    unmarkedData.text[index] = ""
unmarkedData.text = [preprocessing(text) for text in unmarkedData.text]
matrixUnmarked = vectorizer.transform(unmarkedData.text)
matrixUnmarked.shape

C:\Users\Xiaomi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiaomi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Xiaomi\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

(61858, 75869)

In [13]:
nans = markDataFull.text[markDataFull.text.isna()]
for index in nans.index:
    markDataFull.text[index] = ""
markDataFull.text = [preprocessing(text) for text in markDataFull.text]
matrixTarget = vectorizer.transform(markDataFull.text)
matrixTarget.shape

(26510, 75869)

In [27]:
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

svd = TruncatedSVD(n_components=4000, random_state=13)
svd.fit(matrixFullText)

TruncatedSVD(algorithm='randomized', n_components=4000, n_iter=5,
       random_state=13, tol=0.0)

In [28]:
featuresFullText = svd.transform(matrixFullText)
# matrix1 = matrix1.toarray()
featuresFullText.shape

(88368, 4000)

In [29]:
featuresClusterUnique = svd.transform(matrixClusterUnique)
featuresClusterUnique.shape

(3064, 4000)

In [30]:
featuresTarget = svd.transform(matrixTarget)
featuresTarget.shape

(26510, 4000)

In [31]:
featuresUnmarked = svd.transform(matrixUnmarked)
featuresUnmarked.shape

(61858, 4000)

# Попытка кластеризации (не получается метки сопоставить пока)

In [139]:
model = KMeans(init = feautures1, n_clusters=3064, random_state=46)
model.fit(features2)

C:\Users\Xiaomi\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


KMeans(algorithm='auto', copy_x=True,
    init=array([[ 0.01087, -0.00337, ...,  0.00094,  0.01007],
       [ 0.02294, -0.00731, ...,  0.00911,  0.01238],
       ...,
       [ 0.00945, -0.00283, ...,  0.00055, -0.01124],
       [ 0.03841, -0.01168, ..., -0.00266,  0.00247]]),
    max_iter=300, n_clusters=3064, n_init=10, n_jobs=None,
    precompute_distances='auto', random_state=46, tol=0.0001, verbose=0)

In [141]:
preds = model.predict(features)

In [154]:
from collections import Counter

print(Counter(subPreds).most_common(10))
print(Counter(marks).most_common(10))
print(len(indexUnmarkedData))

[(1596, 1119), (1846, 537), (2562, 112), (3034, 90), (2644, 61), (1221, 52), (1760, 49), (160, 47), (423, 45), (415, 44)]
[(103, 399), (238, 190), (67, 178), (21, 171), (195, 167), (637, 157), (5, 140), (311, 131), (169, 130), (25, 118)]
61858


In [153]:
subPreds1 = [preds[index] for index in indexUnmarkedData]
doc_ids = [fullData.doc_id[index] for index in indexUnmarkedData]
with open("C:\\Users\\Xiaomi\\JN\\mail.ru\\homework4\\my_submission.txt", 'w') as outFile:
    outFile.write('doc_id,cat\n')
    for index in range(len(subPreds1)):
        outFile.write(str(doc_ids[index]) +','+str(subPreds1[index]) + '\n')

# Попытка классификации

In [50]:
targets = []

In [61]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression( n_jobs=8)
y = markDataFull.mark.astype ('int')
model.fit(X=featuresTarget,y=y)

C:\Users\Xiaomi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Xiaomi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Xiaomi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=8,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [62]:
pred = model.predict(featuresUnmarked)

In [63]:
pred_valid = model.predict(featuresTarget)

In [54]:
from sklearn.metrics import confusion_matrix
print(type(pred_valid.tolist()))
print(type(markDataFull.mark.to_numpy().tolist()))
confusion_matrix(pred_valid.tolist(), markDataFull.mark.to_numpy().tolist())

<class 'list'>
<class 'list'>


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [64]:
doc_ids = [fullData.doc_id[index] for index in indexUnmarkedData]
with open("C:\\Users\\Xiaomi\\JN\\mail.ru\\homework4\\my_submission.txt", 'w') as outFile:
    outFile.write('doc_id,cat\n')
    for index in range(len(pred)):
        outFile.write(str(doc_ids[index]) +','+str(pred[index]) + '\n')

# Валидация

In [69]:
pred_valid

array([ 238,   57,  425, ..., 1235,  103,    6])

In [73]:
markDataFull.mark.values

array([3059, 57, 425, ..., 1235, 1391, 335], dtype=object)

In [90]:
# print(list(markDataFull[markDataFull.mark == 6].text))

mistakes = [1 if not pred_valid[index] == markDataFull.mark.values[index] else 0 for index in range(len(pred_valid))]

In [93]:
sum(mistakes) / len(pred_valid)

0.43044134288947566

# Еще одна модель

In [102]:
from sklearn.neighbors import KNeighborsClassifier
modelNeigh = KNeighborsClassifier()
modelNeigh.fit(X=featuresTarget,y=y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [108]:
predNeigh = modelNeigh.predict(featuresUnmarked)

In [109]:
predValidNeigh = modelNeigh.predict(featuresTarget)

In [110]:
mistakesNeigh = [1 if not predValidNeigh[index] == markDataFull.mark.values[index] else 0 for index in range(len(predValidNeigh))]

In [111]:
sum(mistakesNeigh) / len(predValidNeigh)

0.1931723877781969

In [112]:
doc_ids = [fullData.doc_id[index] for index in indexUnmarkedData]
with open("C:\\Users\\Xiaomi\\JN\\mail.ru\\homework4\\my_submission.txt", 'w') as outFile:
    outFile.write('doc_id,cat\n')
    for index in range(len(pred)):
        outFile.write(str(doc_ids[index]) +','+str(predNeigh[index]) + '\n')

# Отбор признаков по feature_selection

In [14]:
from sklearn.feature_selection import SelectFromModel
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
modelSelect = SelectFromModel(SGDClassifier(), max_features=12000)
y = markDataFull.mark.astype ('int')
selector = modelSelect.fit(X=matrixTarget, y=y)

C:\Users\Xiaomi\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [15]:
newMatrixTarger = selector.transform(matrixTarget)
newMatrixTarger.shape

(26510, 10914)

In [88]:
from sklearn.naive_bayes import MultinomialNB
modelNB = MultinomialNB()
y = markDataFull.mark.astype ('int')
modelNB.fit(X=newMatrixTarger,y=y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [89]:
predNB = modelNB.predict(selector.transform(matrixUnmarked))
predValidNB = modelNB.predict(selector.transform(matrixTarget))
mistakesNB = [1 if not predValidNB[index] == markDataFull.mark.values[index] else 0 for index in range(len(predValidNB))]
sum(mistakesNB) / len(predValidNB)

0.7640890230101849

In [17]:
from sklearn.linear_model import LogisticRegression
modelLog = SGDClassifier()
modelLog.fit(X=newMatrixTarger,y=y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [19]:
predLog = modelLog.predict(selector.transform(matrixUnmarked))
predValidLog = modelLog.predict(newMatrixTarger)
mistakesLog = [1 if not predValidLog[index] == markDataFull.mark.values[index] else 0 for index in range(len(predValidLog))]
sum(mistakesLog) / len(predValidLog)

0.05322519803847605

In [100]:
doc_ids = [fullData.doc_id[index] for index in indexUnmarkedData]
with open("C:\\Users\\Xiaomi\\JN\\mail.ru\\homework4\\my_submission4.txt", 'w') as outFile:
    outFile.write('doc_id,cat\n')
    for index in range(len(predLog)):
        outFile.write(str(doc_ids[index]) +','+str(predLog[index]) + '\n')

In [ ]:
from sklearn.linear_model import LogisticRegression
modelLog = SGDClassifier()
modelLog.fit(X=newMatrixTarger,y=y)

In [ ]:
predLog = modelLog.predict(selector.transform(matrixUnmarked))
predValidLog = modelLog.predict(newMatrixTarger)
mistakesLog = [1 if not predValidLog[index] == markDataFull.mark.values[index] else 0 for index in range(len(predValidLog))]
sum(mistakesLog) / len(predValidLog)

In [106]:
doc_ids = [fullData.doc_id[index] for index in indexUnmarkedData]
with open("C:\\Users\\Xiaomi\\JN\\mail.ru\\homework4\\my_submission5.txt", 'w') as outFile:
    outFile.write('doc_id,cat\n')
    for index in range(len(predLog)):
        outFile.write(str(doc_ids[index]) +','+str(predLog[index]) + '\n')

In [57]:
from sklearn.neighbors import KNeighborsClassifier
modelKNN = KNeighborsClassifier(n_neighbors = 3)
modelKNN.fit(X=newMatrixTarger,y=y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [58]:
predKNN = modelKNN.predict(selector.transform(matrixUnmarked))
predValidKNN = modelKNN.predict(selector.transform(matrixTarget))
mistakesKNN = [1 if not predValidKNN[index] == markDataFull.mark.values[index] else 0 for index in range(len(predValidKNN))]
sum(mistakesKNN) / len(predValidKNN)

0.1955488494907582

In [65]:
from sklearn.neighbors import KNeighborsClassifier
modelKNN1 = KNeighborsClassifier(n_neighbors = 5)
modelKNN1.fit(X=newMatrixTarger,y=y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [66]:
predKNN1 = modelKNN1.predict(selector.transform(matrixUnmarked))
predValidKNN1 = modelKNN1.predict(selector.transform(matrixTarget))
mistakesKNN1 = [1 if not predValidKNN1[index] == markDataFull.mark.values[index] else 0 for index in range(len(predValidKNN1))]
sum(mistakesKNN1) / len(predValidKNN1)

0.46035458317615996

# Строим ансамбль моделей

In [40]:
with open("my_submission (1).txt") as file1, open("my_submission (2).txt") as file2, open("my_submission (3).txt") as file3, open("my_submission4.txt") as file4, open("my_submission5.txt") as file5, open("my_submission_boost.txt") as file6:
    pred1 = file1.readlines()
    del pred1[0]
    for index in range(len(pred1)):
        pred1[index] = int(pred1[index].split(',')[1].replace('\n', ''))
    pred2 = file2.readlines()
    del pred2[0]
    for index in range(len(pred2)):
        pred2[index] = int(pred2[index].split(',')[1].replace('\n', ''))
    pred3 = file3.readlines()
    del pred3[0]
    for index in range(len(pred3)):
        pred3[index] = int(pred3[index].split(',')[1].replace('\n', ''))
    pred4 = file4.readlines()
    del pred4[0]
    for index in range(len(pred4)):
        pred4[index] = int(pred4[index].split(',')[1].replace('\n', ''))
    pred5 = file5.readlines()
    del pred5[0]
    for index in range(len(pred5)):
        pred5[index] = int(pred5[index].split(',')[1].replace('\n', ''))
    pred6 = file6.readlines()
    del pred6[0]
    for index in range(len(pred6)):
        pred6[index] = int(pred6[index].split(',')[1].replace('\n', ''))

In [41]:
from collections import Counter
pred7 = predLog
for index in range(len(pred7)):
    counter = Counter([pred1[index], pred2[index], pred3[index], pred4[index], pred5[index], pred6[index]])
    if counter.most_common(1)[0][1] >= 2:
        pred7[index] = counter.most_common(1)[0][0]

In [42]:
doc_ids = [fullData.doc_id[index] for index in indexUnmarkedData]
with open("C:\\Users\\Xiaomi\\JN\\mail.ru\\homework4\\my_submission.txt", 'w') as outFile:
    outFile.write('doc_id,cat\n')
    for index in range(len(pred7)):
        outFile.write(str(doc_ids[index]) +','+str(pred7[index]) + '\n')

In [31]:
from sklearn.ensemble import BaggingClassifier

boostModel = BaggingClassifier(SGDClassifier(), n_estimators=40)
boostModel.fit(X=newMatrixTarger,y=y)

C:\Users\Xiaomi\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


BaggingClassifier(base_estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=40, n_jobs=None, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [32]:
predboost = boostModel.predict(selector.transform(matrixUnmarked))
predValidboost = boostModel.predict(newMatrixTarger)
mistakesboost = [1 if not predValidboost[index] == markDataFull.mark.values[index] else 0 for index in range(len(predValidboost))]
sum(mistakesboost) / len(predValidboost)

0.061750282912108635

In [33]:
doc_ids = [fullData.doc_id[index] for index in indexUnmarkedData]
with open("C:\\Users\\Xiaomi\\JN\\mail.ru\\homework4\\my_submission_boost.txt", 'w') as outFile:
    outFile.write('doc_id,cat\n')
    for index in range(len(predboost)):
        outFile.write(str(doc_ids[index]) +','+str(predboost[index]) + '\n')